# [WIP] Magnetic Field Gradient Simulation

This notebook implements a simulation of the gradient of a magnetic field applied to a population of spins.

In [1]:
from pyMRI.pymri import population_transverse_decay
import numpy as np